# Optimizing Online 5g Machine-Learning with Nsight Compute

## 05 Multiple Samples per Block

In this step, the thread group has been partitioned into tiles, so each block can process multiple samples. This is very easy to do once the kernel has been implemented using cooperative groups (which is already the case in this tutorial's starting version). The goal here is to maximize the kernel's occupancy of the GPU, targeting 100% theoretical occupancy.

The optimization has been prepared in the code already. To enable the new version, we just need to set the `APSM_DETECT_VERSION` flag in line 73 of [apsm_versions.h](apsm/cpp/lib/apsm/apsm_versions.h). After setting this define to `apsm_version::APSM_DETECT_SPB`, re-compile the code with the following command:

In [ ]:
%cd /dli/task/ncu/apsm/cpp/build
!make -j

Open [apsm_detect.cu](apsm/cpp/lib/apsm/apsm_detect.cu) at line 400 to inspect the differences between the kernels.

We could profile the optimized kernel the same interactive way as before, but we can also try to use Nsight Compute's `ncu` command line interface (CLI) this time, especially since we know exactly which kernel to profile already. The below command uses the CLI to specify the name of the kernel, the section/metric set to collect, as well as the name of the output report file. We could inspect the results directly on the console, too, but comparing it to the previous version is easier in the UI.

In [ ]:
%cd /dli/task/ncu/apsm/cpp/build
!ncu -k kernel_apsm_detect --set full -f -o /dli/task/ncu/report_spb bin/APSM_tool -m QAM16 -s ../data/offline/rx/time/rxData_QAM16_alltx_converted.bin -r ../data/offline/tx/NOMA_signals_qam16_complex.bin

### Steps without instructor in `...`

After collecting the results, switch to the Ubuntu instance with Nsight Compute and open the just created report file `/root/Desktop/reports/ncu/report_spb.ncu-rep`. After it opened, go back to the first CG report (it may be named "Untitled" if you didn't save it) and activate `Add Baseline` to add the old result as a baseline for comparison. Then, switch again to the new SPB report, which now has all current results compared against the unoptimized baseline data.

<img src="images/ncu_report02_01.png">

The primary thing to notice is that the overall performance, both in runtime, as well as in compute or memory utilization, didn't really change. That's unfortunate, and we need to find out why that is. However, to verify that the change behaves as expected with respect to `occupancy`, let's look at that section first.

<img src="images/ncu_report02_02.png">

As we planned, the occupancies improved with theoretical occupancy now being at 100% GPU maximum. The achieved occupancy and active warps scaled the same. Since improving the occupancy was a means to improve the theoretical warps per scheduler, we should inspect that section, too.

<img src="images/ncu_report02_03.png">

In the `Scheduler Statistics` section, performance improved along with the new occupancy for both theoretical and achieved values. In addition, there is also a 40% improvement in the `eligible warps per scheduler`. Unfortunately, it did not result in a noticeable performance gain for the `issued warps per scheduler` (which ideally should get close to 1 in a perfect scenario).

So why did the issued warps not improve? The reason for the overall performance staying the same across the two versions can be found in the `Warp State Statistics` section: the stalled by instruction have gone up by 90%, largely due to the increase in LG Throttle stalls. This stall kind indicates warps waiting for the L1 instruction queue for local and global (LG) memory operations to be not full. Typically, it occurs only when executing local or global memory instructions extremely frequently.

<img src="images/ncu_report02_04.png">

The Warp States chart only shows us the stall reasons averaged over the runtime of the kernel. To identify in more detail where in the code they occur, we can look at the `Source Counters` section at the end of the Details page:

<img src="images/ncu_report02_05.png">

Besides some statistics on the branching behavior of the code, there are three tables showing the top 5 source locations for stall reasons and executed instructions, respectively. Since warp states per instruction are collected using PC sampling, they are referred to here as `Sampling Data`. The two charts `Sampling Data (All)` and `Sampling Data (Not Issued)` highlight two aspects of such warp states:

* "All" data is increased during collection whenever the sampled scheduler had a stalled warp.
* "Not issued" is increased during collection when the sampled scheduler had a stalled warp, and no other warp could issue instead in the same scheduler slot.

We could say that Sampling Data (All) does not account as well for latency hiding but gives a good picture how many warps in total where stalled, while Sampling Data (Not Issued) highlights specifically the locations where stall latency couldn't be hidden by issuing another warp.

It appears that for this kernel, there is not much of a difference between the two metrics, so we can focus on Sampling Data (All) for simplicity. If you click the link for the source location with the highest metric value (~27% of all samples), you are taken to the `Source` page, and the matching code location is selected.

<img src="images/ncu_report02_06.png">

You will notice that the page currently only shows the kernel assembly (SASS) instructions. That's because we didn't build the kernel with line information, i.e. the compiler didn't embed any correlation information between source and assembly into the binary.

Let's fix the missing line information in our build script, so that we can have proper CUDA-C/SASS correlation. We only need to enable `-lineinfo` in lines 97-99 of [CMakeLists.txt](apsm/cpp/CMakeLists.txt)

```cmake
96 # Lineinfo (--generate-line-info/-lineinfo)
97 #if ( NOT "${CMAKE_CUDA_FLAGS}" MATCHES "-lineinfo" )
98 #    list ( APPEND CUDA_NVCC_FLAGS "-lineinfo" )
99 #endif ()
```

Once that is enabled, re-build the app and re-collect the report. Notice that we also use `--import-source yes` from now on. While this isn't strictly necessary as long as the report is opened on the same system where the code was compiled (so that the file paths embedded in the binary can be found by the tool), it's still helpful in many cases if one wants to move the report around, or preserve the original source with the profile result.

In [ ]:
%cd /dli/task/ncu/apsm/cpp/build
!cmake ..
!make -j
!ncu -k kernel_apsm_detect --set full --import-source yes -f -o /dli/task/ncu/report_spb_lineinfo bin/APSM_tool -m QAM16 -s ../data/offline/rx/time/rxData_QAM16_alltx_converted.bin -r ../data/offline/tx/NOMA_signals_qam16_complex.bin

When opening the new report, we see similar performance data on the `Source` page, but now we can properly correlate it with the high-level code:

<img src="images/ncu_report02_07.png">

Continue optimizations in [step 06](06_shmem.ipynb)